In [ ]:
%pip install -qU supabase
%pip install -qU nltk

### Imports


In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import SupabaseVectorStore

from langchain_ollama import OllamaEmbeddings
from supabase.client import Client, create_client
import nltk

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

load_dotenv()

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

# Ollama embeddings model
embeddings = OllamaEmbeddings(
    model="llama3.1"
)

### Constants


In [3]:
table_name = "documents"
query_name = "match_documents"

### Read from database


In [4]:
vector_store = SupabaseVectorStore(
    embedding=embeddings,
    client=supabase,
    table_name=table_name,
    query_name=query_name,
)

### Initialize the model


In [5]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# Initialize the model
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
)


### Prompt template


In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a software developer specialized in {coding_language}."),
        ("human", "Create a {input_component} React component to be used in an online banking app. Use architectural best practices from {architecture_context}, design style guidelines from {design_context} and business logic inside the component from {user_story_context}. Include code comments. Ensure to use GraphQL, enclose the outermost DIV with the proper TailwindCSS classes and return code only."),
    ]
)


### Invoke chain


In [ ]:
input_component = "Sign Up Form"
coding_language = "React"

# Function to concatenate the vector store results
def concatenate_results(results):
    return " ".join([result.page_content for result in results])

# Architecture context
arch_question = "What are the architectural best practices for a " + input_component + " " + coding_language + " component?"
arch_docs = vector_store.similarity_search(arch_question, k=4, filter={"source": "./data/archguide.html"})
architecture_context = concatenate_results(arch_docs) if arch_docs else None
print("architecture_context: ", architecture_context)
print("\n\n.................................................")

# Design context
design_question = "What are the visual design guidelines for a " + input_component + " " + coding_language + " component?"
design_docs = vector_store.similarity_search(design_question, k=4, filter={"source": "./data/styleguide.md"})
design_context = concatenate_results(design_docs) if design_docs else None
print("design_context: ", design_context)
print("\n\n.................................................")

# User story context
user_story_question = "What are business rules for a " + input_component + " " + coding_language + " component?"
user_story_docs = vector_store.similarity_search(user_story_question, k=4, filter={"source": "./data/userstory.md"})
user_story_context = concatenate_results(user_story_docs) if user_story_docs else None
print("user_story_context: ", user_story_context)
print("\n\n.................................................")

chain = prompt | llm
ai_msg = chain.invoke(
    {
        "input_component": input_component,
        "coding_language": coding_language,
        "architecture_context": architecture_context,
        "design_context": design_context,
        "user_story_context": user_story_context,
    }
)

print("-----------------------------------------------------------------------------------------------------------------")
print("\n\nLLM: ", ai_msg.content)
